In [ ]:
#!pip install fastapi uvicorn

In [ ]:
import uvicorn
from fastapi import FastAPI
import numpy as np
import pandas as pd
import pickle
from pydantic import BaseModel

In [ ]:
description = """
Credit Card Default prediction API helps you to determine who will default in their credit card payments.

# Items

You can **predict** credit card defaulters

# Users

Just enter the fields below in the predict tab to make predictions

All values are to be input as integers

**LIMIT_BAL**: Amount of given credit in NT dollars

**SEX**: Gender (1 = Male, 2 = Female)

**EDUCATION**: (1 = Graduate School, 2 = University, 3 = High school, 4 = Others)

**MARRIAGE**: Marital status (1 = Married, 2 = Single, 3 = Others)

**PAY_1**: Repayment status in September (-1= pay duly, 1= Payment delay for one month, 2= Payment delay for two months,
                                       3= Payment delay for three months, 4= Payment delay for four months, 
                                     … 8= Payment delay for eight months, 9= Payment delay for nine months and above)

**PAY_2**: Repayment status in August (same scale as above)

**PAY_3**: Repayment status in July (same scale as above)

**PAY_4**: Repayment status in June (same scale as above)

**PAY_5**: Repayment status in May (same scale as above)

**PAY_6**: Repayment status in April (same scale as above)

**Total bill amount** : Total amount on bill for 6 months

**Total paid amount** : Total amount paid in 6 months


"""

In [ ]:
# Initializing the app object
app = FastAPI(title= "Credit Card Defaulter Predictor",
              description= description)

In [ ]:
# Opening and loading pickled files
pickle_in = open('svc_model.pkl',"rb")
classifier = pickle.load(pickle_in)
scalar=pickle.load(open('scaling.pkl','rb'))

In [ ]:
@app.get('/')
def index():
    return {'Message': 'Welcome to credit card defaulter prediction. Type "/docs" at the end of the above url'}

In [ ]:
@app.get('/{name}')
def get_name(name: str):
    return {'Hello, Please input your name here': f'{name}'}

In [ ]:
class credits(BaseModel):
    limit_bal: float
    sex: int
    education: int
    marriage : int
    age : int
    pay_1 : int
    pay_2 : int
    pay_3 : int
    pay_4 : int
    pay_5 : int
    pay_6 : int
    total_bill_amount : float
    total_paid_amount : float

In [ ]:
@app.post('/predict')
def predict_defaulter(data: credits):
    data = data.dict()
    limit_bal = data['limit_bal']
    sex = data['sex']
    education = data['education']
    marriage = data['marriage']
    age = data['age']
    pay_1 = data['pay_1']
    pay_2 = data['pay_2']
    pay_3 = data['pay_3']
    pay_4 = data['pay_4']
    pay_5 = data['pay_5']
    pay_6 = data['pay_6']
    total_bill_amount = data['total_bill_amount']
    total_paid_amount = data['total_paid_amount']
    prediction = classifier.predict([[limit_bal, sex, education, marriage, age,
                                      pay_1, pay_2, pay_3, pay_4, pay_5, pay_6,
                                      total_bill_amount, total_paid_amount]])
    print(np.array(list(data.values())).reshape(1,-1))
    scaled_data=scalar.transform(np.array(list(data.values())).reshape(1,-1))
    prediction = classifier.predict(scaled_data)
    if (prediction[0] == 1):
        prediction = "Defaulter"
    else:
        prediction = "Not defaulter"
    return {
        'prediction': prediction
    }
    

In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
if __name__ == '__main__':
    uvicorn.run(app, host = '127.0.0.1', port = 8000)